## Multiple Object Detection

In [58]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/sibylhe/Documents/DR/image_extraction/Image Extraction-eecf9a57849d.json"

def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision_v1p3beta1 as vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))

![title](image/138+E+14th+St,+New+York_1.png)

In [61]:
localize_objects("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_1.png")

Number of objects found: 3

Car (confidence: 0.8524525761604309)
Normalized bounding polygon vertices: 
 - (0.4814865291118622, 0.7513974905014038)
 - (0.9968664050102234, 0.7513974905014038)
 - (0.9968664050102234, 0.9966025948524475)
 - (0.4814865291118622, 0.9966025948524475)

Van (confidence: 0.7084232568740845)
Normalized bounding polygon vertices: 
 - (0.0039339810609817505, 0.590809166431427)
 - (0.1351248323917389, 0.590809166431427)
 - (0.1351248323917389, 0.9920144081115723)
 - (0.0039339810609817505, 0.9920144081115723)

Building (confidence: 0.5453587174415588)
Normalized bounding polygon vertices: 
 - (0.00035295262932777405, 0.0012941285967826843)
 - (0.9996469616889954, 0.0012941285967826843)
 - (0.9996469616889954, 0.8899999856948853)
 - (0.00035295262932777405, 0.8899999856948853)


“Car” and “Van” constitute 18% of the image

![title](image/138+E+14th+St,+New+York_2.png)

In [62]:
localize_objects("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_2.png")

Number of objects found: 2

Van (confidence: 0.9696447849273682)
Normalized bounding polygon vertices: 
 - (0.3594490885734558, 0.5237245559692383)
 - (0.9996097087860107, 0.5237245559692383)
 - (0.9996097087860107, 0.9190990924835205)
 - (0.3594490885734558, 0.9190990924835205)

Car (confidence: 0.8812007308006287)
Normalized bounding polygon vertices: 
 - (0.0010600239038467407, 0.6170122623443604)
 - (0.18224647641181946, 0.6170122623443604)
 - (0.18224647641181946, 0.9102446436882019)
 - (0.0010600239038467407, 0.9102446436882019)


“Van” and “Car” constitute 40% of the image.

![title](image/138+E+14th+St,+New+York_6.png)

In [63]:
localize_objects("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_6.png")

Number of objects found: 2

Bicycle (confidence: 0.6480509042739868)
Normalized bounding polygon vertices: 
 - (0.5916005969047546, 0.7851858139038086)
 - (0.7204241156578064, 0.7851858139038086)
 - (0.7204241156578064, 0.9436563849449158)
 - (0.5916005969047546, 0.9436563849449158)

Building (confidence: 0.6258305907249451)
Normalized bounding polygon vertices: 
 - (0.0008235424757003784, 0.0022353047970682383)
 - (0.9991764426231384, 0.0022353047970682383)
 - (0.9991764426231384, 0.8899999856948853)
 - (0.0008235424757003784, 0.8899999856948853)


![title](image/138+E+14th+St,+New+York_5.png)

In [66]:
localize_objects("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_5.png")

Number of objects found: 1

Building (confidence: 0.5453587174415588)
Normalized bounding polygon vertices: 
 - (0.00035295262932777405, 0.0017647221684455872)
 - (0.9991764426231384, 0.0017647221684455872)
 - (0.9991764426231384, 0.8899999856948853)
 - (0.00035295262932777405, 0.8899999856948853)


## OCR

In [67]:
import io
def detect_handwritten_ocr(path):
    """Detects handwritten characters in a local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision_v1p3beta1 as vision
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    # Language hint codes for handwritten OCR:
    # en-t-i0-handwrit, mul-Latn-t-i0-handwrit
    # Note: Use only one language hint code per request for handwritten OCR.
    image_context = vision.types.ImageContext(
        language_hints=['en-t-i0-handwrit'])

    response = client.document_text_detection(image=image,
                                              image_context=image_context)

    print('Full Text: {}'.format(response.full_text_annotation.text))
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))

![title](image/11620+Monument+Dr,+Fairfax,+VA+22030.png)

In [71]:
detect_handwritten_ocr("/Users/sibylhe/Documents/DR/image_extraction/image/11620+Monument+Dr,+Fairfax,+VA+22030.png")

Full Text: Wegmans
ÇADA A


Block confidence: 0.8799999952316284

Paragraph confidence: 0.8799999952316284
Word text: Wegmans (confidence: 0.8799999952316284)
	Symbol: W (confidence: 0.75)
	Symbol: e (confidence: 0.9599999785423279)
	Symbol: g (confidence: 0.9800000190734863)
	Symbol: m (confidence: 0.9300000071525574)
	Symbol: a (confidence: 0.8999999761581421)
	Symbol: n (confidence: 0.9399999976158142)
	Symbol: s (confidence: 0.7599999904632568)

Block confidence: 0.4300000071525574

Paragraph confidence: 0.4300000071525574
Word text: ÇADA (confidence: 0.4699999988079071)
	Symbol: Ç (confidence: 0.23999999463558197)
	Symbol: A (confidence: 0.3499999940395355)
	Symbol: D (confidence: 0.3799999952316284)
	Symbol: A (confidence: 0.9300000071525574)
Word text: A (confidence: 0.28999999165534973)
	Symbol: A (confidence: 0.28999999165534973)


![title](image/138+E+14th+St,+New+York_5.png)

In [70]:
detect_handwritten_ocr("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_5.png")

Full Text: IT TT TTT
TTTTT
RANDO


Block confidence: 0.550000011920929

Paragraph confidence: 0.550000011920929
Word text: IT (confidence: 0.49000000953674316)
	Symbol: I (confidence: 0.4000000059604645)
	Symbol: T (confidence: 0.5799999833106995)
Word text: TT (confidence: 0.550000011920929)
	Symbol: T (confidence: 0.8899999856948853)
	Symbol: T (confidence: 0.2199999988079071)
Word text: TTT (confidence: 0.5899999737739563)
	Symbol: T (confidence: 0.5600000023841858)
	Symbol: T (confidence: 0.6499999761581421)
	Symbol: T (confidence: 0.5799999833106995)

Block confidence: 0.6100000143051147

Paragraph confidence: 0.6100000143051147
Word text: TTTTT (confidence: 0.6100000143051147)
	Symbol: T (confidence: 0.9200000166893005)
	Symbol: T (confidence: 0.5)
	Symbol: T (confidence: 0.6800000071525574)
	Symbol: T (confidence: 0.3400000035762787)
	Symbol: T (confidence: 0.6100000143051147)

Block confidence: 0.8399999737739563

Paragraph confidence: 0.8399999737739563
Word text: RANDO (confi

![title](image/138+E+14th+St,+New+York_6.png)

In [72]:
detect_handwritten_ocr("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_6.png")

Full Text: TTTTTTTT
TRADERO
MHESHO


Block confidence: 0.7900000214576721

Paragraph confidence: 0.7900000214576721
Word text: TTTTTTTT (confidence: 0.7900000214576721)
	Symbol: T (confidence: 0.8199999928474426)
	Symbol: T (confidence: 0.7300000190734863)
	Symbol: T (confidence: 0.8199999928474426)
	Symbol: T (confidence: 0.8799999952316284)
	Symbol: T (confidence: 0.9200000166893005)
	Symbol: T (confidence: 0.6499999761581421)
	Symbol: T (confidence: 0.8700000047683716)
	Symbol: T (confidence: 0.699999988079071)

Block confidence: 0.8199999928474426

Paragraph confidence: 0.8399999737739563
Word text: TRADERO (confidence: 0.8399999737739563)
	Symbol: T (confidence: 0.949999988079071)
	Symbol: R (confidence: 0.9700000286102295)
	Symbol: A (confidence: 0.9800000190734863)
	Symbol: D (confidence: 0.9399999976158142)
	Symbol: E (confidence: 0.9100000262260437)
	Symbol: R (confidence: 0.5600000023841858)
	Symbol: O (confidence: 0.5899999737739563)
Paragraph confidence: 0.800000011920929
W

![title](image/138+E+14th+St,+New+York_1.png)

In [73]:
detect_handwritten_ocr("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_1.png")

Full Text: PLA
I
IR DAR
FER THAT
KDEN JEE
WINE SHOP
EAST COAST


Block confidence: 0.6700000166893005

Paragraph confidence: 0.6700000166893005
Word text: PLA (confidence: 0.6700000166893005)
	Symbol: P (confidence: 0.8199999928474426)
	Symbol: L (confidence: 0.3799999952316284)
	Symbol: A (confidence: 0.8199999928474426)

Block confidence: 0.5099999904632568

Paragraph confidence: 0.5099999904632568
Word text: I (confidence: 0.5099999904632568)
	Symbol: I (confidence: 0.5099999904632568)

Block confidence: 0.550000011920929

Paragraph confidence: 0.550000011920929
Word text: IR (confidence: 0.5699999928474426)
	Symbol: I (confidence: 0.44999998807907104)
	Symbol: R (confidence: 0.6899999976158142)
Word text: DAR (confidence: 0.5400000214576721)
	Symbol: D (confidence: 0.5)
	Symbol: A (confidence: 0.6499999761581421)
	Symbol: R (confidence: 0.49000000953674316)

Block confidence: 0.5899999737739563

Paragraph confidence: 0.5899999737739563
Word text: FER (confidence: 0.7599999904632568

![title](image/138+E+14th+St,+New+York_2.png)

In [74]:
detect_handwritten_ocr("/Users/sibylhe/Documents/DR/image_extraction/image/138+E+14th+St,+New+York_2.png")

Full Text: 7 TEREZA
TRADERS
NE SHOP
& EC MECHANICA
East Coast Mechanical Contracting C
EAST COAST
1800 545-9155
INDUSTRIAL CÒMMERCIAL BOLER & BURNER INSTA


Block confidence: 0.6800000071525574

Paragraph confidence: 0.6800000071525574
Word text: 7 (confidence: 0.7599999904632568)
	Symbol: 7 (confidence: 0.7599999904632568)
Word text: TEREZA (confidence: 0.6600000262260437)
	Symbol: T (confidence: 0.5600000023841858)
	Symbol: E (confidence: 0.7400000095367432)
	Symbol: R (confidence: 0.949999988079071)
	Symbol: E (confidence: 0.5400000214576721)
	Symbol: Z (confidence: 0.5899999737739563)
	Symbol: A (confidence: 0.6299999952316284)

Block confidence: 0.8700000047683716

Paragraph confidence: 0.8799999952316284
Word text: TRADERS (confidence: 0.8799999952316284)
	Symbol: T (confidence: 0.9900000095367432)
	Symbol: R (confidence: 0.9900000095367432)
	Symbol: A (confidence: 1.0)
	Symbol: D (confidence: 0.9300000071525574)
	Symbol: E (confidence: 0.9399999976158142)
	Symbol: R (confidence: